In [9]:
#header files
import pandas as pd
import timeit
from copy import deepcopy
import multiprocessing
import os
import json
from sklearn.model_selection import KFold

# task 2, header files
from gensim.models import Word2Vec,KeyedVectors
from sklearn.neural_network import MLPClassifier
import numpy as np
import timeit
from gensim.scripts.glove2word2vec import glove2word2vec

In [10]:
#reading the file

start = timeit.default_timer()   # to note time
filename = "Brown_tagged_train.txt"

print("Reading the file..........\n")
f = open(filename, "r")
maintxt = f.read()
sentences = list(map(str,maintxt.split("\n")))
for i in range(len(sentences)-1,-1,-1):
    if len(sentences[i])< 2:
        sentences.pop(i)

Reading the file..........



In [11]:
#3 functions

def breakWordTag(s):
    if s.find("/") > 0:
        l  = s.split("/")
        return " ".join(l[:-1]) , l[-1]
    return None,None

# def formVocabDict(wor):          
#     if wor in vocab:
#         vocab[wor] += 1
#     else:
#         vocab[wor] = 1

def formRequiredTextTaggedList(orig_text,orig_tagged,indices):
    temp_text_list = []
    temp_tagged_list = []
    for index in indices:
        temp_text_list.append(orig_text[index])
        temp_tagged_list.append(orig_tagged[index])
    return temp_text_list, temp_tagged_list


# def updateTagAndWordDict(word,tag):
#     if tag in tag_dict:
#         if word in tag_dict[tag]:
#             tag_dict[tag][word] += 1
#         else:
#             tag_dict[tag][word] = 1
#         tag_count[tag] += 1
#         formVocabDict(word)
#     else:
#         tag_dict[tag] = {word:1}
#         tag_count[tag] = 1

def formTransitionMatrix(tag_count,tagged_sent):    
    transition_matrix = {"ST1":{}}     #transition matrix
    for i in tag_count:
        transition_matrix[i] = {}

    for i in transition_matrix:
        for j in tag_count:
            transition_matrix[i][j] = 0
        transition_matrix[i]["END1"] = 0

    for tag_sent in tagged_sent:
        transition_matrix["ST1"][tag_sent[0]] += 1
        for j in range(1,len(tag_sent)):
            transition_matrix[tag_sent[j-1]][tag_sent[j]] += 1
        transition_matrix[tag_sent[-1]]["END1"] += 1

    for i in transition_matrix:
        for j in transition_matrix[i]:
            transition_matrix[i][j] += 1           #SMOOTHING
    for i in transition_matrix:
        coun = 0
        for j in transition_matrix[i]:
            coun += transition_matrix[i][j]
        coun = max(1,coun)
        for j in transition_matrix[i]:
            transition_matrix[i][j] /= coun
    return transition_matrix

def getEmissionProb(word,tag,tag_dict,vocab):         # to give relative emission probability
    if word not in vocab:      #SMOOTHING
        return 1
    if word in tag_dict[tag]:
        return tag_dict[tag][word]
    return 0

# def remover():
#     rem = 5                   # tune your speed lesser is higher
#     if len(removlist) > rem:
#         removlist.sort(reverse = True)
#         ele = removlist[rem]
#         for i in range(len(que2)-1,-1,-1):
#             if que2[i] < ele:
#                 que1.pop(i)
#                 que2.pop(i)
                
def precall(ke,tagwise_metric,average):
    fn = tagwise_metric[ke]["fn"]
    fp = tagwise_metric[ke]["fp"]
    tp = tagwise_metric[ke]["tp"]
    try:
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1s = 2/((1/recall)+(1/precision))
        average[0].append(precision)
        average[1].append(recall)
        average[2].append(f1s)
    except:
        precision = 0
        recall = 0
        f1s = 0	

    tagwise_metric[ke]["precision"] = precision
    tagwise_metric[ke]["recall"] = recall
    tagwise_metric[ke]["f1s"] = f1s
    return tagwise_metric, average

In [12]:
#defining some holders

# tag_dict = {}              # tag dictionary (Emision matrix)
# tag_count = {}             # count accurance of tags
# vocab ={}                  # vocabulary
# word_dict = {}             # word dictionary

text_sent = []             # contain list of sentencess
tagged_sent = []           # contain list of ist of tagged sentencess

testingsen = []            # contain testing sentencess
testingtag = []            # contain testing tags

senten_length = []
  

In [13]:
print("Breaking into sentencess and tags..........\n")    

for i in sentences:  
    ss = list(map(str,i.split()))       # process sentencess
    temp1 = []
    temp2 = []
    for j in ss:
        wor,tag = breakWordTag(j)
        if wor != None:
            temp1.append(wor)
            temp2.append(tag)
    if len(temp1) > 1:                # remove lesser length sentencess
        text_sent.append(temp1)
        tagged_sent.append(temp2)
        senten_length.append(len(temp1))

orig_text_sent = text_sent
orig_tagged_sent = tagged_sent

Breaking into sentencess and tags..........



In [23]:
#optional

tag_dict = {}              # tag dictionary (Emision matrix)
tag_count = {}             # count accurance of tags
vocab ={}                  # vocabulary
word_dict = {}             # word dictionary

#forming word and tagdict
for i in range(len(text_sent)):           
    for j in range(len(text_sent[i])):
        word, tag = text_sent[i][j] , tagged_sent[i][j]
        if word not in word_dict:
            word_dict[word] = {}
        if tag in word_dict[word]:
            word_dict[word][tag] += 1
        else:
            word_dict[word][tag]=1

        if tag in tag_dict:
            if word in tag_dict[tag]:
                tag_dict[tag][word] += 1
            else:
                tag_dict[tag][word] = 1
            tag_count[tag] += 1
        else:
            tag_dict[tag] = {word:1}
            tag_count[tag] = 1
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1

KeyError: 'VERB'

In [20]:
print("\n\nWords having more than 1 tag")
for i in word_dict:
    if len(word_dict[i]) > 1:
        print(i , word_dict[i])



Words having more than 1 tag
that {'DET': 1095, 'ADP': 2789, 'PRON': 670, 'ADV': 33}
time {'NOUN': 776, 'VERB': 1}
to {'PRT': 6763, 'ADP': 5089, 'X': 1}
a {'DET': 9819, 'X': 2, 'NOUN': 1}
for {'ADP': 3898, 'ADV': 2}
such {'ADJ': 431, 'PRT': 142, 'ADV': 9}
matter {'NOUN': 117, 'VERB': 20}
of {'ADP': 15791, 'X': 2}
state {'NOUN': 243, 'VERB': 6}
Once {'ADP': 14, 'ADV': 36}
principle {'NOUN': 43, 'ADJ': 1}
increase {'NOUN': 76, 'VERB': 37}
in {'ADP': 8321, 'PRT': 278, 'X': 1}
reasons {'NOUN': 38, 'VERB': 1}
need {'NOUN': 64, 'VERB': 66}
1 {'NUM': 344, 'NOUN': 1}
below {'ADV': 43, 'ADP': 45}
shows {'VERB': 38, 'NOUN': 1}
loose {'ADJ': 19, 'VERB': 1, 'ADV': 2}
But {'CONJ': 610, 'ADP': 1}
it {'PRON': 3376, 'PRT': 1}
show {'VERB': 101, 'NOUN': 17}
over {'ADP': 419, 'PRT': 207, 'ADJ': 3}
In {'ADP': 813, 'PRT': 1}
past {'ADJ': 45, 'NOUN': 36, 'ADP': 38, 'ADV': 8}
then {'ADV': 562, 'ADJ': 10}
there {'PRT': 656, 'ADV': 341}
many {'ADJ': 317, 'ADV': 4, 'PRT': 7}
as {'ADP': 2505, 'ADV': 444}
use 

Severe {'ADJ': 2, 'NOUN': 1}
collapse {'NOUN': 2, 'VERB': 1}
shock {'NOUN': 16, 'VERB': 1}
narrowing {'NOUN': 1, 'VERB': 2}
brown {'ADJ': 39, 'NOUN': 1}
neck {'NOUN': 54, 'VERB': 2}
swollen {'ADJ': 7, 'VERB': 4}
oval {'ADJ': 2, 'NOUN': 2}
staining {'VERB': 13, 'NOUN': 23}
sweet {'ADJ': 53, 'ADV': 1}
extract {'NOUN': 1, 'VERB': 3}
stem {'NOUN': 18, 'VERB': 1}
adhesive {'NOUN': 3, 'ADJ': 2}
Black {'NOUN': 23, 'ADJ': 7}
dark {'ADJ': 112, 'NOUN': 14}
stress {'NOUN': 78, 'VERB': 4}
signs {'NOUN': 21, 'VERB': 1}
fight {'NOUN': 21, 'VERB': 15}
restoring {'VERB': 1, 'NOUN': 1}
comment {'VERB': 2, 'NOUN': 7}
obscure {'VERB': 1, 'ADJ': 6}
tuning {'NOUN': 2, 'VERB': 1}
neurotic {'ADJ': 4, 'NOUN': 1}
cure {'VERB': 4, 'NOUN': 4}
polynomial {'NOUN': 27, 'ADJ': 1}
span {'VERB': 4, 'NOUN': 2}
null {'ADJ': 9, 'NOUN': 3}
substitute {'NOUN': 4, 'ADJ': 2, 'VERB': 3}
shooting {'VERB': 5, 'NOUN': 5}
hit {'VERB': 51, 'NOUN': 4}
dice {'NOUN': 10, 'VERB': 1}
shot {'VERB': 24, 'NOUN': 39}
toss {'NOUN': 2, 'VERB

In [ ]:
# This is to reduce the set and speed up the process, use it for MLP mostly

all_tags = {}
for tag_sen in orig_tagged_sent:
    for tag in tag_sen:
        if tag in all_tags:
            all_tags[tag]+=1
        else:
            all_tags[tag]=1

n = len(orig_text_sent)
reduction_factor = 10
newn = int(n/reduction_factor)
orig_text_sent = orig_text_sent[:newn]
orig_tagged_sent = orig_tagged_sent[:newn]

# Task 1 - HMM

In [25]:
#perform k fold 
k = 3
n = len(orig_text_sent)
kf = KFold(n_splits=k)



foldi = 1
#accuracy, precision, recall, f1 score
total_metric = [[],[],[],[]]

incorrect_classifications = []

for train_indices,test_indices in kf.split(orig_text_sent):
    testingsen = []
    testingtag = []
    
    testingsen , testingtag = formRequiredTextTaggedList(orig_text_sent,orig_tagged_sent,test_indices)
    text_sent , tagged_sent = formRequiredTextTaggedList(orig_text_sent,orig_tagged_sent,train_indices)
    
    print('Length of test set is '+str(len(test_indices)))
    print('Length of train set is '+str(len(text_sent)))
    
    tag_dict = {}              # tag dictionary (Emision matrix)
    tag_count = {}             # count accurance of tags
    vocab ={}                  # vocabulary
    word_dict = {}             # word dictionary
    
    #forming word and tagdict
    for i in range(len(text_sent)):           
        for j in range(len(text_sent[i])):
            word, tag = text_sent[i][j] , tagged_sent[i][j]
            if word not in word_dict:
                word_dict[word] = {}
            if tag in word_dict[word]:
                word_dict[word][tag] += 1
            else:
                word_dict[word][tag]=1

            if tag in tag_dict:
                if word in tag_dict[tag]:
                    tag_dict[tag][word] += 1
                else:
                    tag_dict[tag][word] = 1
                tag_count[tag] += 1
            else:
                tag_dict[tag] = {word:1}
                tag_count[tag] = 1
            if word in vocab:
                vocab[word] += 1
            else:
                vocab[word] = 1
    
    #normalizing tag_dict
    for i in tag_dict:
        for j in tag_dict[i]:
            tag_dict[i][j] /= tag_count[i]
    
    #transition matrix
    transition_matrix = formTransitionMatrix(tag_count,tagged_sent)
    
    acctemp = []

    #connfusion matrix
    confusion_matrix = {}
    for tag in tag_count:
        confusion_matrix[tag] = {}
        for tag2 in tag_count:
            confusion_matrix[tag][tag2] = 0
    
    #tagwise precision matrix
    tagwise_metric = {}    
    for tag in tag_count:
        tagwise_metric[tag] = {"tp":0,"fp":0,"fn":0}
    
    
    errors = []
    
    # Testing start
    for test_text_ind in range(len(testingsen)):  
        test_text = testingsen[test_text_ind]
        que1 = []
        que2 = []

        for tag in transition_matrix["ST1"]:
            if  tag != "END1" and transition_matrix["ST1"][tag] * getEmissionProb(test_text[0],tag,tag_dict,vocab) > 0:
                que1.append([tag])
                que2.append(transition_matrix["ST1"][tag] )
        
        for tword_ind in range(1,len(test_text)):
            t1 = []
            t2 = []
            removlist = []

            while(que1):
                temp1 = que1.pop()
                temp2 = que2.pop()

                for tag in tag_count:
                    sss = transition_matrix[temp1[-1]][tag] * getEmissionProb(test_text[tword_ind],tag,tag_dict,vocab) * temp2
                    if  sss > 0:
                        t1.append(temp1 + [tag])
                        t2.append(sss)
                        removlist.append(sss)
            que1 = t1
            que2 = t2
#             remover()
            rem = 5                   # tune your speed lesser is higher
            if len(removlist) > rem:
                removlist.sort(reverse = True)
                ele = removlist[rem]
                for i in range(len(que2)-1,-1,-1):
                    if que2[i] < ele:
                        que1.pop(i)
                        que2.pop(i)
        
        incorrect = False
        incorrect_sample = {'tags':[]}
        try:
            anspos = que2.index(max(que2))
            accuracy = 0
            for ttag_ind in range(len(testingtag[test_text_ind])):
                if testingtag[test_text_ind][ttag_ind] == que1[anspos][ttag_ind]:
                    confusion_matrix[que1[anspos][ttag_ind]][que1[anspos][ttag_ind]] += 1
                    tagwise_metric[que1[anspos][ttag_ind]]["tp"] += 1
                    accuracy += 1
                else:
                    incorrect = True
                    incorrect_sample['tags'].append([ttag_ind,testingtag[test_text_ind][ttag_ind],que1[anspos][ttag_ind]])
                    confusion_matrix[testingtag[test_text_ind][ttag_ind]][que1[anspos][ttag_ind]] += 1
                    tagwise_metric[que1[anspos][ttag_ind]]["fp"] += 1
                    tagwise_metric[testingtag[test_text_ind][ttag_ind]]["fn"] += 1
            if incorrect:
                incorrect_sample['text'] = test_text
                incorrect_classifications.append(incorrect_sample)
            accuracy = (accuracy / len(testingtag[test_text_ind])) * 100
            acctemp.append(accuracy)
        except:
            pass
        
    print('Accuracy for fold '+str(foldi)+ " is " +str(sum(acctemp)/len(acctemp)))
    
    folder_path = 'HMM_Fold'+str(foldi)
    try:
        os.mkdir(folder_path)
    except:
        # the folder is already created
        pass
       
    df = pd.DataFrame(confusion_matrix)
    df.to_csv(folder_path+"/confusionmatrixbi.csv")
    
    print("\n\nNumber of words for each tag :")
    for i in tag_dict:
        print(i,len(tag_dict[i]))

    tagfile = open(folder_path+'/noofwordsinTag.txt','wt')
    tagfile.write(str(tag_count))
    tagfile.close()
    
#     print("\n\nWords having more than 1 tag")
#     for i in word_dict:
#         if len(word_dict[i]) > 1:
#             print(i , word_dict[i])
    
    average = [[],[],[]] 
    for i in tagwise_metric:
        tagwise_metric, average = precall(i,tagwise_metric,average)

    df = pd.DataFrame(tagwise_metric)
    df.to_csv(folder_path+"/tagwisebi.csv")
    
    print("\nAverage precision = ", sum(average[0])/len(average[0]))
    print("Average recall = ", sum(average[1])/len(average[1]))
    print("Average f1 Score = ", sum(average[2])/len(average[2]))
    print("Total words present = ",len(vocab))
    print("Total tags present = ",len(tag_count))
    print()
    
    total_metric[0].append(sum(acctemp)/len(acctemp))
    total_metric[1].append(sum(average[0])/len(average[0]))
    total_metric[2].append(sum(average[1])/len(average[1]))
    total_metric[3].append(sum(average[2])/len(average[2]))
    
    foldi+=1
#     break



# with open('HMM_incorrect_classifications', 'w') as fout:
#     json.dump(incorrect_classifications, fout)

f  = open('HMM_incorrect_classifications.txt', 'w')
f.write("No of incorrect classifications are " + str(len(incorrect_classifications)))
f.write('\n\n')
for icc in incorrect_classifications:
    f.write(str(icc))
    f.write('\n\n')
f.close()

print('\nOverall Statistics')
print('Final Accuracy = ',sum(total_metric[0])/len(total_metric[0]))
print('Final Precision = ',sum(total_metric[1])/len(total_metric[1]))
print('Final Recall = ',sum(total_metric[2])/len(total_metric[2]))
print('Final F1 Score = ',sum(total_metric[3])/len(total_metric[3]))

print("Total sentencess = ",  len(senten_length))
print("Maximum length sentences = ", max(senten_length))
print("Minimum length sentences = ",min(senten_length))
print("Average length of sentences= ", sum(senten_length)/len(senten_length))

Length of test set is 917
Length of train set is 1832
Accuracy for fold 1 is 89.91166522140115


Number of words for each tag :
VERB 1628
. 14
DET 45
NOUN 3657
ADP 110
ADJ 1068
NUM 279
ADV 401
CONJ 14
PRT 20
PRON 22
X 29

Average precision =  0.8605335090502044
Average recall =  0.8435213699279083
Average f1 Score =  0.8482734738758854
Total words present =  6946
Total tags present =  12

Length of test set is 916
Length of train set is 1833
Accuracy for fold 2 is 90.46462502711297


Number of words for each tag :
ADP 112
DET 47
NOUN 3724
VERB 1574
ADJ 1127
CONJ 13
PRT 19
. 11
ADV 382
NUM 293
PRON 24
X 17

Average precision =  0.9187450536860283
Average recall =  0.8864293578926611
Average f1 Score =  0.8999148694677693
Total words present =  7018
Total tags present =  12

Length of test set is 916
Length of train set is 1833
Accuracy for fold 3 is 89.2278626959015


Number of words for each tag :
ADP 110
DET 50
NOUN 3392
VERB 1573
ADJ 1013
CONJ 13
PRT 19
. 14
ADV 381
NUM 327
PRON 26
X

# Task 2 - MLP Classifier

In [6]:
# word2vecmodel = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
# word2vecmodel = Word2Vec(orig_text_sent,min_count=1,size=32)

glove_input_file = 'glove.6B/glove.6B.50d.txt'
word2vec_output_file = 'word2vecfromGlove.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
word2vecmodel  = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [11]:
k = 3
n = len(orig_text_sent)
kf = KFold(n_splits=k)



foldi = 1
#accuracy, precision, recall, f1 score
total_metric = [[],[],[],[]]

incorrect_classifications = []

def createFeatures(word2vecmodel,text_sen,tag_sen,size,vec_size=32):
    tempX = []
    tempY = [] 
    for i in range(size):
        for j in range(len(text_sen[i])):
            try:
                tempX.append(word2vecmodel.wv[text_sen[i][j]])
            except:
                tempX.append(np.zeros(vec_size).astype(float))
            tempY.append(tag_sen[i][j])
    return tempX,tempY


for train_indices,test_indices in kf.split(orig_text_sent):
    testingsen = []
    testingtag = []
    
    testingsen , testingtag = formRequiredTextTaggedList(orig_text_sent,orig_tagged_sent,test_indices)
    text_sent , tagged_sent = formRequiredTextTaggedList(orig_text_sent,orig_tagged_sent,train_indices)
    
    print('Length of test set is '+str(len(test_indices)))
    print('Length of train set is '+str(len(text_sent)))
           
    tag_count = {}             
    vocab ={}
    word_dict={}
    
    #forming word and tagdict
    for i in range(len(text_sent)):           
        for j in range(len(text_sent[i])):
            word, tag = text_sent[i][j] , tagged_sent[i][j]
            if word not in word_dict:
                word_dict[word] = {}
            if tag in word_dict[word]:
                word_dict[word][tag] += 1
            else:
                word_dict[word] = {tag:1}
            if tag in tag_count:
                tag_count[tag]+=1
            else:
                tag_count[tag]=1
            if word in vocab:
                vocab[word] += 1
            else:
                vocab[word] = 1
    
    acctemp = []

    #connfusion matrix
    confusion_matrix = {}
    for tag in all_tags:
        confusion_matrix[tag] = {}
        for tag2 in all_tags:
            confusion_matrix[tag][tag2] = 0
    
    #tagwise precision matrix
    tagwise_metric = {}    
    for tag in all_tags:
        tagwise_metric[tag] = {"tp":0,"fp":0,"fn":0}
    
    # vec_size = change for glove, for word2vec its 32
    vec_size = 50
    X_train, Y_train = createFeatures(word2vecmodel,text_sent,tagged_sent,len(text_sent),vec_size)
    X_test,Y_test = createFeatures(word2vecmodel,testingsen,testingtag,len(testingsen),vec_size)
    
    model = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
    
    model.fit(X_train,Y_train)
    
    Y_predict = model.predict(X_test)
    
    index=0
    for i in range(len(testingsen)):
        incorrect = False
        incorrect_sample = {'tags':[]}
        accuracy=0
        for j in range(len(testingsen[i])):
            predicted_tag = Y_predict[index]
            true_tag = testingtag[i][j]
#             Y_test[index]
            if true_tag==predicted_tag:
                accuracy+=1
                confusion_matrix[true_tag][predicted_tag] += 1
                tagwise_metric[true_tag]["tp"] += 1
            else:
                incorrect = True
                incorrect_sample['tags'].append([j,true_tag,predicted_tag])
                confusion_matrix[true_tag][predicted_tag] += 1
                tagwise_metric[predicted_tag]["fp"] += 1
                tagwise_metric[true_tag]["fn"] += 1
            index+=1
        
        if incorrect:
            incorrect_sample['text'] = testingsen[i]
            incorrect_classifications.append(incorrect_sample)
        
        accuracy = (accuracy / len(testingtag[i])) * 100
        acctemp.append(accuracy)
        
    print('Accuracy for fold '+str(foldi)+ " is " +str(sum(acctemp)/len(acctemp)))
    
    folder_path = 'MLP_Fold'+str(foldi)
    try:
        os.mkdir(folder_path)
    except:
        # the folder is already created
        pass
       
    df = pd.DataFrame(confusion_matrix)
    df.to_csv(folder_path+"/confusionmatrixbi.csv")
    
    print("\n\nNumber of words for each tag :")
    for i in tag_count:
        print(i,tag_count[i])

    tagfile = open(folder_path+'/noofwordsinTag.txt','wt')
    tagfile.write(str(tag_count))
    tagfile.close()
    
#     print("\n\nWords having more than 1 tag")
#     for i in word_dict:
#         if len(word_dict[i]) > 1:
#             print(i , word_dict[i])
    
    average = [[],[],[]] 
    for i in tagwise_metric:
        tagwise_metric, average = precall(i,tagwise_metric,average)

    df = pd.DataFrame(tagwise_metric)
    df.to_csv(folder_path+"/tagwisebi.csv")
    
    print("\nAverage precision = ", sum(average[0])/len(average[0]))
    print("Average recall = ", sum(average[1])/len(average[1]))
    print("Average f1 Score = ", sum(average[2])/len(average[2]))
    print()
    
    total_metric[0].append(sum(acctemp)/len(acctemp))
    total_metric[1].append(sum(average[0])/len(average[0]))
    total_metric[2].append(sum(average[1])/len(average[1]))
    total_metric[3].append(sum(average[2])/len(average[2]))
    
    foldi+=1
    
#     break

    
f  = open('MLP_incorrect_classifications.txt', 'w')
f.write("No of incorrect classifications are " + str(len(incorrect_classifications)))
f.write('\n\n')
for icc in incorrect_classifications:
    f.write(str(icc))
    f.write('\n\n')
f.close()

print('\nOverall Statistics')
print('Final Accuracy = ',sum(total_metric[0])/len(total_metric[0]))
print('Final Precision = ',sum(total_metric[1])/len(total_metric[1]))
print('Final Recall = ',sum(total_metric[2])/len(total_metric[2]))
print('Final F1 Score = ',sum(total_metric[3])/len(total_metric[3]))

print("Total sentencess = ",  len(senten_length))
print("Maximum length sentences = ", max(senten_length))
print("Minimum length sentences = ",min(senten_length))
print("Average length of sentences= ", sum(senten_length)/len(senten_length))

Length of test set is 917
Length of train set is 1832


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Accuracy for fold 1 is 87.94720887616538


Number of words for each tag :
VERB 7219
. 4810
DET 6115
NOUN 13805
ADP 7436
ADJ 3948
NUM 1030
ADV 1822
CONJ 1675
PRT 931
PRON 866
X 32

Average precision =  0.8990469067134993
Average recall =  0.8490502413484152
Average f1 Score =  0.8619649133029417

Length of test set is 916
Length of train set is 1833


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Accuracy for fold 2 is 87.96114915461186


Number of words for each tag :
ADP 7327
DET 6024
NOUN 14268
VERB 6968
ADJ 4135
CONJ 1701
PRT 883
. 5170
ADV 1620
NUM 1194
PRON 553
X 20

Average precision =  0.9266282502924746
Average recall =  0.8057532567455385
Average f1 Score =  0.8399530062774048

Length of test set is 916
Length of train set is 1833


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Accuracy for fold 3 is 87.33944519787639


Number of words for each tag :
ADP 7625
DET 6031
NOUN 14179
VERB 7337
ADJ 4073
CONJ 1886
PRT 1000
. 5692
ADV 1692
NUM 1350
PRON 835
X 32

Average precision =  0.9198993602424587
Average recall =  0.8045212454586096
Average f1 Score =  0.8353043499629091


Overall Statistics
Final Accuracy =  87.74926774288456
Final Precision =  0.9151915057494775
Final Recall =  0.8197749145175212
Final F1 Score =  0.8457407565144185
Total sentencess =  27491
Maximum length sentences =  386
Minimum length sentences =  2
Average length of sentences=  19.756502127969153


In [66]:
# # It is not feasible to run trigram in notebook because it takes 10 - 20 GB of data 
# # I ran trigram on iiit Delhi server which has 50 GB of RAM and 20 cores of CPU


# transition_matrixtri = {str(["ST1","ST2"]):{}}   # making of transition matrix for trigram
# for i in tag_count:
#     transition_matrixtri[str(["ST2",i])] = {}

# for i in tag_count:
#     for j in tag_count:
#         transition_matrixtri[str([i,j])] = {}

# for i in transition_matrixtri:
#     for j in tag_count:
#         transition_matrixtri[i][j] = 1                     # SMOOTHING

# for i in tagged_sent:
#     try:
#         transition_matrixtri[str(["ST1","ST2"])][i[0]] += 1
#         transition_matrixtri[str(["ST2",i[0]])][i[1]] += 1
#     except:
#         print(i)

# for i in tagged_sent:
#     for j in range(2,len(i)):
#         transition_matrixtri[str([i[j-2],i[j-1]])][i[j]] += 1

# for i in transition_matrixtri:
#     coun = 0
#     for j in transition_matrixtri[i]:
#         coun += transition_matrixtri[i][j]

#     coun = max(1,coun)
#     for j in transition_matrixtri[i]:
#         transition_matrixtri[i][j] /= coun



In [ ]:
# start = timeit.default_timer()
# acctemp = []


# confusion_matrix = {}          # confusion matrix for trigrams
# for i in tag_count:
#     confusion_matrix[i] = {}
#     for j in tag_count:
#         confusion_matrix[i][j] = 0

# tagwise_metric = {}        # tagwise precision,recall for trigrams
# for i in tag_count:
#     tagwise_metric[i] = {"tp":0,"fp":0,"fn":0}

In [ ]:
# print("Testing start .........")
# for test in range(len(testingsen)):         # test data to be trained
#     test_text = testingsen[test] 

#     que1 = []
#     que2 = []

#     for i in transition_matrixtri[str(["ST1","ST2"])]:
#         if  i != "END1" and transition_matrixtri[str(["ST1","ST2"])][i] * getEmissionProb(test_text[0],i) > 0:
#             que1.append([i])
#             que2.append(transition_matrixtri[str(["ST1","ST2"])][i])

#     for i in range(1,len(test_text)):
#         t1 = []
#         t2 = []
#         removlist = []

#         while(que1):
#             temp1 = que1.pop()
#             temp2 = que2.pop()
#             if len(temp1) == 1:
#                 for j in tag_count:
#                     sss = transition_matrixtri[str(["ST2",temp1[-1]])][j] * getEmissionProb(test_text[i],j) * temp2
#                     if  sss > 0:
#                         t1.append(temp1 + [j])
#                         t2.append(sss)
#                         removlist.append(sss)
#             else:
#                 for j in tag_count:
#                     sss = transition_matrixtri[str([temp1[-2],temp1[-1]])][j] * getEmissionProb(test_text[i],j) * temp2
#                     if  sss > 0:
#                         t1.append(temp1 + [j])
#                         t2.append(sss)
#                         removlist.append(sss)


#         que1 = t1
#         que2 = t2

#         remover()

#     try:           # try catch to handle errors SMOOTHING
#         anspos = que2.index(max(que2))
#         accuracy = 0
#         for i in range(len(testingtag[test])):
#             if testingtag[test][i] == que1[anspos][i]:
#                 confusion_matrix[que1[anspos][i]][que1[anspos][i]] += 1
#                 tagwise_metric[que1[anspos][i]]["tp"] += 1
#                 accuracy += 1
#             else:
#                 confusion_matrix[testingtag[test][i]][que1[anspos][i]] += 1
#                 tagwise_metric[que1[anspos][i]]["fp"] += 1
#                 tagwise_metric[testingtag[test][i]]["fn"] += 1

#         accuracy = accuracy / len(testingtag[test]) * 100
#         print(test+1,"Accuracy =",accuracy,"%")
#         acctemp.append(accuracy)

#     except:
#         pass


#     #print("\n\n\n")



In [9]:
# average = [[],[],[]]      # contain average for precion,recall, F1-score
# def precall(ke): 
#     fn = tagwise_metric[ke]["fn"]
#     fp = tagwise_metric[ke]["fp"]
#     tp = tagwise_metric[ke]["tp"]
#     try:
#         precision = tp/(tp+fp)
#         recall = tp/(tp+fn)
#         f1s = 2/((1/recall)+(1/precision))
#         average[0].append(precision)
#         average[1].append(recall)
#         average[2].append(f1s)
#     except:
#         precision = 0
#         recall = 0
#         f1s = 0	

#     tagwise_metric[ke]["precision"] = precision
#     tagwise_metric[ke]["recall"] = recall
#     tagwise_metric[ke]["f1s"] = f1s


#     #print("\n\n\n")

# df = pd.DataFrame(confusion_matrix)
# df.to_csv("confusionmatrixtri.csv")




# print("\n\nNumber of words for each tag :")
# for i in tag_dict:
#     print(i,len(tag_dict[i]))

# print("\n\nWords having more than 1 tag")
# for i in word_dict:
#     if len(word_dict[i]) > 1:
#         print(i , word_dict[i])

# for i in tagwise_metric:
#     precall(i)

# df = pd.DataFrame(tagwise_metric)
# df.to_csv("tagwisetri.csv")

# print("\nAverage precision = ", sum(average[0])/len(average[0]))
# print("Average recall = ", sum(average[1])/len(average[1]))
# print("Average f1 Score = ", sum(average[2])/len(average[2]))

# print("Total words present = ",len(vocab))
# print("Total tags present = ",len(tag_count))
# print("Total sentencess = ",  len(senten_length))
# print("Maximum length sentences = ", max(senten_length))
# print("Minimum length sentences = ",min(senten_length))
# print("Average length of sentences= ", sum(senten_length)/len(senten_length))

# stop = timeit.default_timer()

# print("\nAverage accuracy = ", sum(acctemp)/len(acctemp))
# print('Time: ', stop - start) 

# Thank You!